In [ ]:
print('this is first init for jupyter notebook')

In [1]:
pwd

'/content'

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
print('dari colab')

dari colab


In [5]:
cd drive/MyDrive/git-colab-repo/

/content/drive/MyDrive/git-colab-repo


In [ ]:
!git status

In [7]:
cd movie-genre-classification/

/content/drive/MyDrive/git-colab-repo/movie-genre-classification
